In [1]:
import os
import pandas as pd
import qiime2 as q2
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
data_dir = 'Alien_data'

In [3]:
metadata=pd.read_csv('Alien_data/sample_metadata.tsv', sep='\t')

In [4]:
metadata['alleged_abduction']= metadata['alleged_abduction'].values.astype(str)
metadata['alleged_abduction'] = metadata['alleged_abduction'].str.replace('0','non_abducted')
metadata['alleged_abduction'] = metadata['alleged_abduction'].str.replace('1','abducted')
metadata['week_relative_to_hct']= metadata['week_relative_to_hct'].values.astype(str)
metadata['week_relative_to_hct']= metadata['week_relative_to_hct'].str.replace('-1.0','one week before HCT', regex=False)
metadata['week_relative_to_hct']= metadata['week_relative_to_hct'].str.replace('-2.0','two weeks before HCT', regex=False)
metadata['week_relative_to_hct']= metadata['week_relative_to_hct'].str.replace('1.0','one week after HCT', regex=False)
metadata['week_relative_to_hct']= metadata['week_relative_to_hct'].str.replace('2.0','two weeks after HCT', regex=False)
metadata['week_relative_to_hct']= metadata['week_relative_to_hct'].str.replace('0.0','HCT week', regex=False)
metadata['week_relative_to_hct']= metadata['week_relative_to_hct'].str.replace('nan','', regex=False)

In [47]:
metadata.to_csv('Alien_data/str_metadata.tsv', sep='\t', encoding='utf-8', escapechar='\n', index=False)

In [6]:
# get an overview of the feature table:
! qiime feature-table summarize \
  --i-table $data_dir/table-filtered.qza \
  --m-sample-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/feature_table_filtered.qzv

Saved Visualization to: Alien_data/feature_table_filtered.qzv


In [7]:
Visualization.load(f'{data_dir}/feature_table_filtered.qzv')

<visualization: Visualization uuid: f96d9c6f-a919-4951-860a-78efc462fdf7>

### **Alpha rarefaction using the fragment insert tree:**

In [9]:
# show rarefaction plot (we can also try to increase the depth) 
! qiime diversity alpha-rarefaction \
    --i-table $data_dir/table-filtered.qza \
    --i-phylogeny $data_dir/sepp-tree.qza \
    --p-max-depth 10000 \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --o-visualization $data_dir/alpha-rarefaction.qzv

Saved Visualization to: Alien_data/alpha-rarefaction.qzv


In [10]:
Visualization.load(f'{data_dir}/alpha-rarefaction.qzv')

<visualization: Visualization uuid: 323ba8ef-a095-4dc0-be8f-01956f365f9e>

### Alpha rarefaction using the de novo tree (an alternative but not suggested):

In [11]:
# A slightly different plot by using the bootstraped tree, but the trends are similar:
#! qiime diversity alpha-rarefaction \
#    --i-table $data_dir/table-filtered.qza \
#    --i-phylogeny $data_dir/raxml-cat-bootstrap-rooted.qza \
#    --p-max-depth 10000 \
#    --m-metadata-file $data_dir/sample_metadata.tsv \
#   --o-visualization $data_dir/alpha-rarefaction-denovo.qzv

In [12]:
# Visualization.load(f'{data_dir}/alpha-rarefaction-denovo.qzv')

### **Diversity analysis**

In [13]:
# sampling depth of 3000 is chosen (we can discuss which is the best depth to chose)
# 4000?
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/table-filtered.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien_data/core-metrics-results/weighted_unifrac_pcoa_results.qza
Saved PCo

**Test the associations between categorical metadata columns with the corresponding diversity metrics**: 

In [7]:
# faith_pd
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/faith-pd-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results/faith-pd-group-significance.qzv


In [8]:
Visualization.load(f'{data_dir}/core-metrics-results/faith-pd-group-significance.qzv')

<visualization: Visualization uuid: 86274458-bdfb-4b1e-ac49-e4be4f90f4f3>

In [9]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results/observed-features-group-significance.qzv


In [10]:
Visualization.load(f'{data_dir}/core-metrics-results/observed-features-group-significance.qzv')

<visualization: Visualization uuid: cac424c7-f7e6-4b73-acfe-44a0c46d5212>

In [11]:
# Shannon
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results/shannon_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/shannon-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results/shannon-group-significance.qzv


In [12]:
Visualization.load(f'{data_dir}/core-metrics-results/shannon-group-significance.qzv')

<visualization: Visualization uuid: fe3d04a7-a370-4744-8177-03355df2a571>

In [13]:
# evenness
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results/evenness_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/evenness-vector-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results/evenness-vector-group-significance.qzv


In [14]:
Visualization.load(f'{data_dir}/core-metrics-results/evenness-vector-group-significance.qzv')

<visualization: Visualization uuid: ca50358f-009c-4834-9853-b50e8017d4e3>

**Next, test whether numeric sample metadata columns (day_relative_to_nearest_hct) are correlated with microbial community**:

In [15]:
#faith_pd
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results/faith_pd_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/faith-pd-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results/faith-pd-group-significance-numeric.qzv


In [16]:
Visualization.load(f'{data_dir}/core-metrics-results/faith-pd-group-significance-numeric.qzv')

<visualization: Visualization uuid: 80745a42-c3c7-491a-b752-88528e9f6bc7>

In [17]:
# observed_features
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/observed-features-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results/observed-features-group-significance-numeric.qzv


In [18]:
Visualization.load(f'{data_dir}/core-metrics-results/observed-features-group-significance-numeric.qzv')

<visualization: Visualization uuid: bde51555-a404-4ab3-a4da-28867c66f7a3>

In [19]:
# shannon
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results/shannon_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/shannon-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results/shannon-group-significance-numeric.qzv


In [20]:
Visualization.load(f'{data_dir}/core-metrics-results/shannon-group-significance-numeric.qzv')

<visualization: Visualization uuid: c0ea85ff-fef2-4ac3-91a8-ccfa4807e5a3>

In [21]:
# evenness
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results/evenness_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results/evenness-vector-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results/evenness-vector-group-significance-numeric.qzv


In [22]:
Visualization.load(f'{data_dir}/core-metrics-results/evenness-vector-group-significance-numeric.qzv')

<visualization: Visualization uuid: 27e9f4ef-e4e8-4352-af6a-1578125253c9>

* It seems like there are two trends of the change of alpha diversity before and after hct, so we subsetted the metadata `day` column and feature table samples into two categories: `before hct` and `after hct`:

In [15]:
# before hct
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[day_relative_to_nearest_hct]<'0' or [day_relative_to_nearest_hct]='0'" \
    --o-filtered-table $data_dir/before_hct_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/before_hct_feature_table.qza


In [23]:
before_hct = pd.DataFrame(data=metadata[metadata['day_relative_to_nearest_hct'] <= 0])
before_hct.to_csv('Alien_data/before_hct_metadata.tsv', sep='\t', encoding='utf-8', escapechar='\n', index=False)

In [33]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/before_hct_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/before_hct_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-before-hct

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-before-hct/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-before-hct/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-before-hct/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-before-hct/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-before-hct/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-before-hct/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-before-hct/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-before-hct/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-before-hct/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-before-hct/unweighted_unifrac_pcoa

In [34]:
# faith_pd
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-before-hct/faith_pd_vector.qza \
  --m-metadata-file $data_dir/before_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-before-hct/faith-pd-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-before-hct/faith-pd-group-significance-numeric.qzv


In [35]:
Visualization.load(f'{data_dir}/core-metrics-results-before-hct/faith-pd-group-significance-numeric.qzv')

<visualization: Visualization uuid: 135acac2-ec46-4093-b346-3e79a3317276>

In [36]:
# observed_features
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-before-hct/observed_features_vector.qza \
  --m-metadata-file $data_dir/before_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-before-hct/observed-features-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-before-hct/observed-features-group-significance-numeric.qzv


In [19]:
Visualization.load(f'{data_dir}/core-metrics-results-before-hct/observed-features-group-significance-numeric.qzv')

<visualization: Visualization uuid: e6bf483e-e39b-4676-84a7-ba6080d053b9>

In [38]:
# shannon
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-before-hct/shannon_vector.qza \
  --m-metadata-file $data_dir/before_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-before-hct/shannon-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-before-hct/shannon-group-significance-numeric.qzv


In [39]:
Visualization.load(f'{data_dir}/core-metrics-results-before-hct/shannon-group-significance-numeric.qzv')

<visualization: Visualization uuid: 7750a955-f276-4cdd-8765-be54915081e4>

In [40]:
# evenness
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-before-hct/evenness_vector.qza \
  --m-metadata-file $data_dir/before_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-before-hct/evenness-vector-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-before-hct/evenness-vector-group-significance-numeric.qzv


In [41]:
Visualization.load(f'{data_dir}/core-metrics-results-before-hct/evenness-vector-group-significance-numeric.qzv')

<visualization: Visualization uuid: 52bd6455-5c60-41a7-b59c-99dfe22d9d38>

In [24]:
# after hct
after_hct = pd.DataFrame(data=metadata[metadata['day_relative_to_nearest_hct'] >= 0])
after_hct.to_csv('Alien_data/after_hct_metadata.tsv', sep='\t', encoding='utf-8', escapechar='\n', index=False)
after_hct

,sampleid,stool_consistency,hct_source,disease,categorical_time_relative_to_engraftment,week_relative_to_hct,timepoint_of_transplant,day_relative_to_nearest_hct,alleged_abduction
2,5Y49IM,semi-formed,cord,Leukemia,peri,one week before HCT,7.0,0.0,abducted
3,ZKJI45,semi-formed,cord,Leukemia,post,one week after HCT,7.0,8.0,non_abducted
4,2I7SIQ,liquid,cord,Leukemia,peri,one week before HCT,0.0,0.0,abducted
5,PCUMU7,semi-formed,cord,Leukemia,post,two weeks after HCT,0.0,16.0,abducted
7,AFG7YZ,semi-formed,cord,Leukemia,post,two weeks after HCT,1.0,15.0,non_abducted
...,...,...,...,...,...,...,...,...,...
144,RE2G74,liquid,cord,Hodgkin's Disease,post,one week after HCT,6.0,9.0,non_abducted
145,K613DJ,semi-formed,cord,Hodgkin's Disease,post,one week after HCT,6.0,13.0,non_abducted
148,0WG67D,semi-formed,cord,Leukemia,post,two weeks after HCT,1.0,16.0,abducted
151,95NCMG,liquid,cord,Non-Hodgkin's Lymphoma,peri,one week before HCT,5.0,0.0,non_abducted


In [17]:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[day_relative_to_nearest_hct]>'0' or [day_relative_to_nearest_hct]='0'" \
    --o-filtered-table $data_dir/after_hct_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/after_hct_feature_table.qza


In [44]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/after_hct_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/after_hct_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-after-hct

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-after-hct/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-after-hct/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-after-hct/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-after-hct/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-after-hct/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-after-hct/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-after-hct/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-after-hct/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-after-hct/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-after-hct/unweighted_unifrac_pcoa_results.q

In [45]:
# faith_pd
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-after-hct/faith_pd_vector.qza \
  --m-metadata-file $data_dir/after_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-after-hct/faith-pd-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-after-hct/faith-pd-group-significance-numeric.qzv


In [18]:
Visualization.load(f'{data_dir}/core-metrics-results-after-hct/faith-pd-group-significance-numeric.qzv')

<visualization: Visualization uuid: 52076ce6-42ea-4037-9dc4-bdefb6ab1a1e>

In [47]:
# observed_features
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-after-hct/observed_features_vector.qza \
  --m-metadata-file $data_dir/after_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-after-hct/observed-features-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-after-hct/observed-features-group-significance-numeric.qzv


In [48]:
Visualization.load(f'{data_dir}/core-metrics-results-after-hct/observed-features-group-significance-numeric.qzv')

<visualization: Visualization uuid: 4319e521-a521-4cbc-bc73-377e78c2cc2e>

In [49]:
# shannon
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-after-hct/shannon_vector.qza \
  --m-metadata-file $data_dir/after_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-after-hct/shannon-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-after-hct/shannon-group-significance-numeric.qzv


In [50]:
Visualization.load(f'{data_dir}/core-metrics-results-after-hct/shannon-group-significance-numeric.qzv')

<visualization: Visualization uuid: 83c7a918-2534-4bde-bac7-d27e3dec7408>

In [51]:
# evenness
! qiime diversity alpha-correlation \
  --i-alpha-diversity $data_dir/core-metrics-results-after-hct/evenness_vector.qza \
  --m-metadata-file $data_dir/after_hct_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-after-hct/evenness-vector-group-significance-numeric.qzv

Saved Visualization to: Alien_data/core-metrics-results-after-hct/evenness-vector-group-significance-numeric.qzv


In [52]:
Visualization.load(f'{data_dir}/core-metrics-results-after-hct/evenness-vector-group-significance-numeric.qzv')

<visualization: Visualization uuid: 1f51c2cb-6d54-4948-8334-eb1b1f52651d>

#### **Compare between abducted and non-abuducted patients with different HCT time**

**`Pre`-engraftment**

In [8]:
# pre-engraftment patients:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[categorical_time_relative_to_engraftment]='pre' " \
    --o-filtered-table $data_dir/pre_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/pre_feature_table.qza


In [9]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/pre_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-pre

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-pre/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pre/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pre/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pre/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pre/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pre/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pre/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pre/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pre/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-pre/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien_data/core-metrics-results-pre

In [12]:
# faith_pd
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-pre/faith_pd_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-pre/faith-pd-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-pre/faith-pd-group-significance.qzv


In [15]:
Visualization.load(f'{data_dir}/core-metrics-results-pre/faith-pd-group-significance.qzv')

<visualization: Visualization uuid: 6377c66e-1d64-4489-b3d4-371ca52a806c>

In [16]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-pre/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-pre/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-pre/observed-features-group-significance.qzv


In [17]:
Visualization.load(f'{data_dir}/core-metrics-results-pre/observed-features-group-significance.qzv')

<visualization: Visualization uuid: 2b481352-ee87-4bbd-8a1d-2eb028a5c70e>

In [18]:
# Shannon
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-pre/shannon_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-pre/shannon-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-pre/shannon-group-significance.qzv


In [19]:
Visualization.load(f'{data_dir}/core-metrics-results-pre/shannon-group-significance.qzv')

<visualization: Visualization uuid: 7886d34a-d1e3-45f2-81e9-beb558a47848>

In [20]:
# evenness
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-pre/evenness_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-pre/evenness-vector-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-pre/evenness-vector-group-significance.qzv


In [21]:
Visualization.load(f'{data_dir}/core-metrics-results-pre/evenness-vector-group-significance.qzv')

<visualization: Visualization uuid: 35c3946c-e2cb-4a91-985c-1cfa6a5b5d64>

**`peri`**

In [22]:
# peri-engraftment patients:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[categorical_time_relative_to_engraftment]='peri' " \
    --o-filtered-table $data_dir/peri_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/peri_feature_table.qza


In [23]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/peri_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-peri

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-peri/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-peri/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-peri/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-peri/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-peri/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-peri/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-peri/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-peri/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-peri/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-peri/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien_data/core-metrics-r

In [24]:
# faith_pd
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-peri/faith_pd_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-peri/faith-pd-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-peri/faith-pd-group-significance.qzv


In [25]:
Visualization.load(f'{data_dir}/core-metrics-results-peri/faith-pd-group-significance.qzv')

<visualization: Visualization uuid: 7c665f68-2da6-4cfd-ad03-b68877c843e9>

In [26]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-peri/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-peri/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-peri/observed-features-group-significance.qzv


In [27]:
Visualization.load(f'{data_dir}/core-metrics-results-peri/observed-features-group-significance.qzv')

<visualization: Visualization uuid: a98cbd48-ec32-44d9-9a4d-06cda4140a36>

In [28]:
# Shannon
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-peri/shannon_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-peri/shannon-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-peri/shannon-group-significance.qzv


In [29]:
Visualization.load(f'{data_dir}/core-metrics-results-peri/shannon-group-significance.qzv')

<visualization: Visualization uuid: 3fc69ed5-b395-4c7e-bd98-81ff0968e81f>

In [30]:
# evenness
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-peri/evenness_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-peri/evenness-vector-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-peri/evenness-vector-group-significance.qzv


In [31]:
Visualization.load(f'{data_dir}/core-metrics-results-peri/evenness-vector-group-significance.qzv')

<visualization: Visualization uuid: 34b2bdc7-0417-421c-9f4a-45ce499fff96>

**`post`-engraftment**

In [32]:
# post-engraftment patients:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[categorical_time_relative_to_engraftment]='post' " \
    --o-filtered-table $data_dir/post_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/post_feature_table.qza


In [33]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/post_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-post

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-post/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-post/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-post/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-post/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-post/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-post/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-post/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-post/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-post/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-post/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien_data/core-metrics-r

In [34]:
# faith_pd
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-post/faith_pd_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-post/faith-pd-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-post/faith-pd-group-significance.qzv


In [35]:
Visualization.load(f'{data_dir}/core-metrics-results-post/faith-pd-group-significance.qzv')

<visualization: Visualization uuid: d8790e39-e0f7-4c8e-9759-39ad01a83074>

In [36]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-post/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-post/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-post/observed-features-group-significance.qzv


In [37]:
Visualization.load(f'{data_dir}/core-metrics-results-post/observed-features-group-significance.qzv')

<visualization: Visualization uuid: c2b98fa4-c55a-4cf1-b783-a056dde7d0f4>

In [38]:
# Shannon
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-post/shannon_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-post/shannon-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-post/shannon-group-significance.qzv


In [39]:
Visualization.load(f'{data_dir}/core-metrics-results-post/shannon-group-significance.qzv')

<visualization: Visualization uuid: 82fe1d76-82ce-4cf2-9ed4-a990eede5457>

In [40]:
# evenness
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-post/evenness_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-post/evenness-vector-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-post/evenness-vector-group-significance.qzv


In [41]:
Visualization.load(f'{data_dir}/core-metrics-results-post/evenness-vector-group-significance.qzv')

<visualization: Visualization uuid: 5e728de8-3c69-4ec7-8e4c-44b9d377a62f>

#### **Compare between abducted and non-abuducted patients with different diseases**

**`leukemia`**

In [42]:
# Patients with leukemia
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[disease]='Leukemia' " \
    --o-filtered-table $data_dir/leuk_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/leuk_feature_table.qza


In [72]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/leuk_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-leuk

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-leuk/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-leuk/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-leuk/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-leuk/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-leuk/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-leuk/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-leuk/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-leuk/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-leuk/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-leuk/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien_data/core-metrics-r

In [73]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-leuk/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-leuk/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-leuk/observed-features-group-significance.qzv


In [74]:
Visualization.load(f'{data_dir}/core-metrics-results-leuk/observed-features-group-significance.qzv')

<visualization: Visualization uuid: 33331a10-c989-47d5-916f-50cd08aff262>

In [48]:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[disease]='Non-Hodgkin's Lymphoma' " \
    --o-filtered-table $data_dir/lym_feature_table.qza

There was an issue with loading the file /str_metadata.tsv as metadata:

  Metadata file path doesn't exist, or the path points to something other than a file. Please check that the path exists, has read permissions, and points to a regular file (not a directory): /str_metadata.tsv

  There may be more errors present in the metadata file. To get a full report, sample/feature metadata files can be validated with Keemei: https://keemei.qiime2.org

  Find details on QIIME 2 metadata requirements here: https://docs.qiime2.org/2022.2/tutorials/metadata/



In [ ]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/leuk_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-leuk

#### **Compare between abducted and non-abuducted patients with different hct source**

**`PBSC_unmodified`**

In [49]:
# PBSC_unmodified hct source samples
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[hct_source]='PBSC_unmodified' " \
    --o-filtered-table $data_dir/pbsc_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/pbsc_feature_table.qza


In [50]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/pbsc_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-pbsc

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-pbsc/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pbsc/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pbsc/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pbsc/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-pbsc/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pbsc/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pbsc/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pbsc/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-pbsc/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-pbsc/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien_data/core-metrics-r

In [51]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-pbsc/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-pbsc/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-pbsc/observed-features-group-significance.qzv


In [52]:
Visualization.load(f'{data_dir}/core-metrics-results-pbsc/observed-features-group-significance.qzv')

<visualization: Visualization uuid: 9c4b3d0c-0bdc-45b8-a75b-d1e28711367f>

**`cord`**

In [53]:
# cord hct source samples:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[hct_source]='cord' " \
    --o-filtered-table $data_dir/cord_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/cord_feature_table.qza


In [54]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/cord_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-cord

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-cord/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-cord/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-cord/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-cord/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-cord/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-cord/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-cord/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-cord/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-cord/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-cord/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien_data/core-metrics-r

In [55]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-cord/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-cord/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-cord/observed-features-group-significance.qzv


In [56]:
Visualization.load(f'{data_dir}/core-metrics-results-cord/observed-features-group-significance.qzv')

<visualization: Visualization uuid: 712e3764-91e0-4f74-ab05-b86101c98a1e>

#### **Compare between abducted and non-abuducted patients with different `stool_consistency`**

In [57]:
# formed stool_consistency patients:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[stool_consistency]='formed' " \
    --o-filtered-table $data_dir/formed_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/formed_feature_table.qza


In [58]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/formed_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-formed

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-formed/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-formed/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-formed/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-formed/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-formed/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-formed/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-formed/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-formed/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-formed/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-formed/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien

In [59]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-formed/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-formed/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-formed/observed-features-group-significance.qzv


In [60]:
Visualization.load(f'{data_dir}/core-metrics-results-formed/observed-features-group-significance.qzv')

<visualization: Visualization uuid: 9da2f685-0443-464e-ba25-6ec054ed3e08>

In [61]:
# semi-formed stool_consistency patients:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[stool_consistency]='semi-formed' " \
    --o-filtered-table $data_dir/semi-formed_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/semi-formed_feature_table.qza


In [65]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/semi-formed_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-semi-formed

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-semi-formed/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-semi-formed/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-semi-formed/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-semi-formed/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-semi-formed/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-semi-formed/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-semi-formed/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-semi-formed/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-semi-formed/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-semi-formed/unweighted_un

In [66]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-semi-formed/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-semi-formed/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-semi-formed/observed-features-group-significance.qzv


In [67]:
Visualization.load(f'{data_dir}/core-metrics-results-semi-formed/observed-features-group-significance.qzv')

<visualization: Visualization uuid: 1339d6f8-b03d-4db7-8c26-46081279c0d8>

In [68]:
# liquid stool_consistency patients:
! qiime feature-table filter-samples \
    --i-table $data_dir/table-filtered.qza \
    --m-metadata-file $data_dir/str_metadata.tsv \
    --p-where "[stool_consistency]='liquid' " \
    --o-filtered-table $data_dir/liquid_feature_table.qza

Saved FeatureTable[Frequency] to: Alien_data/liquid_feature_table.qza


In [69]:
! qiime diversity core-metrics-phylogenetic \
  --i-table $data_dir/liquid_feature_table.qza \
  --i-phylogeny $data_dir/sepp-tree.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --p-sampling-depth 3000 \
  --output-dir $data_dir/core-metrics-results-liquid

Saved FeatureTable[Frequency] to: Alien_data/core-metrics-results-liquid/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-liquid/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-liquid/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-liquid/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: Alien_data/core-metrics-results-liquid/evenness_vector.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-liquid/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-liquid/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-liquid/jaccard_distance_matrix.qza
Saved DistanceMatrix to: Alien_data/core-metrics-results-liquid/bray_curtis_distance_matrix.qza
Saved PCoAResults to: Alien_data/core-metrics-results-liquid/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: Alien

In [70]:
# observed features
! qiime diversity alpha-group-significance \
  --i-alpha-diversity $data_dir/core-metrics-results-liquid/observed_features_vector.qza \
  --m-metadata-file $data_dir/str_metadata.tsv \
  --o-visualization $data_dir/core-metrics-results-liquid/observed-features-group-significance.qzv

Saved Visualization to: Alien_data/core-metrics-results-liquid/observed-features-group-significance.qzv


In [71]:
Visualization.load(f'{data_dir}/core-metrics-results-liquid/observed-features-group-significance.qzv')

<visualization: Visualization uuid: 0007e410-c415-480c-b15f-cf813bd0ce7c>